## Meta-commands (about magics)

In [ ]:
%lsmagic

In [ ]:
%automagic

In [ ]:
%magic

In [ ]:
%quickref

Magics come in _line_ and _cell_ flavors.
* Line magics affect only one line within a cell
* Cell magics affect everything inside the cell

In [ ]:
%system ls | head -5
print("The %system magic only affects the first line, so this cell runs correctly.\nMy Python command however eclipses system output. See what happens if you comment me out.")

In [ ]:
%%bash
ls | head -5
print("As Python code, I will break this cell. Run this cell with and without commenting me out.")
echo; echo 'Hello'

## Environment magics
These are useful for querying and manipulating both the shell environment in which a notebook is running, and the notebook environment itself.

In [ ]:
# What directory am I in?
%pwd

In [ ]:
# What's in this directory?
%ls

In [ ]:
# Let's find just the non-hidden directories in this one with a Bash one-liner.
#  Magics can be used on the right side of an assignment.
mydirs_a = %sx find . -maxdepth 1 -type d -not -name '.*'
print(mydirs_a[0:2])

Shell environment variables can be queried and set via `%env`. However, the `%set_env` magic may make your intent clearer.

In [ ]:
# List all environment variables. Produces a dict
%env

In [ ]:
# Set with multiple values provided, using an optional equals sign; query specifically by leaving it out.
%env GOOBERS1=true
%set_env GOOBERS2=/arbitrary/path
%env GOOBERS1 GOOBERS2 # Doesn't query two variables, rather sets the first equal to everything after it.
%env GOOBERS1

Although these are line magics, they have effects for all following cells.
The `%precision` magic is a convenience magic with a similar property.

In [ ]:
a = 1/3.
%precision 3
a

In [ ]:
b = 2/3.
b

Note that this has no effect on print functions or the underlying floating point representation.

In [ ]:
print(a)
print(b)

In [ ]:
# Leave off the argument to reset
%precision
a

## Development Magics - Environment

The `%history` magic can be useful for seeing the order of operations executed in a notebook since its instance inception. This might be helpful in troubleshooting the current state of variables, since cells can be executed out of order as one develops.

The `-n` option adds line numbers and allows selection of a range of lines, and `-f` permits capture to a file. 

In [ ]:
%history

You might just want to see the state of variables, rather than the path by which they got there.
* `%who` lists names as a string
* `%who_ls` lists names as a Python list
* `%whos` provides names, types, and metadata (or values if a simple variable)

The query can filter by variable type.

In [ ]:
%who

In [ ]:
%who_ls

In [ ]:
%whos

In [ ]:
%whos float

If you're developing and find that a set of variables has gotten corrupted in the process, the `%reset` magic is a quicker way of blowing everything away to start over than restarting the kernel. A subtler hand may use the `%reset_selective` magic to just delete variables according to a regex pattern match.

In [ ]:
%reset_selective -f "^a"

In [ ]:
%whos

## Development Magics - Integration and Profiling

Several magics exist for pulling in pieces of external code, as well as timing routines.

For STEM, the `%pylab` magic pulls in numpy (as namespaces `numpy` and `np`), matplotlib (as namespaces `matplotlib` and `mpl`), and matplotlib.pyplot (as namespaces `matplotlib.pyplot`, `pyplot` and `plt`).

In [ ]:
%pylab

See if there is an interesting file containing functions.

In [ ]:
%sx ls MM.py

See what function names are contained in that file.

In [ ]:
%sx grep def MM.py

Bring in the array_multiply function for profiling

In [ ]:
%load -s array_multiply MM.py

In [ ]:
A = np.random.rand(200,100)
B = np.random.rand(100,200)

In [ ]:
%timeit C = array_multiply(A,B)

Let's try to speed this up with Numba. First see if we have it installed. If not, `conda install -y numba`

In [ ]:
%conda list numba

Can now re-define the function that we brought in via the `%load` magic by pasting in after the numba decorator.

In [ ]:
from numba import jit
@jit(nopython=True)
# Paste in function from 4 cells above.

In [ ]:
%timeit D = array_multiply(A,B)

Let's compare that to a numpy matrix multiplication.

In [ ]:
# %load -s matrix_multiply MM.py
def matrix_multiply(a, b):
	"""Convert input arrays to two numpy matrices and use Numpy internals to multiply."""
	assert np.shape(a)[1] == np.shape(b)[0]
	a_p = np.asmatrix(a)
	b_p = np.asmatrix(b)
	return a_p * b_p


In [ ]:
%timeit E = matrix_multiply(A,B)

Note that although the above timings were written as assignments, no assignment is done. `%timeit` just times the function run, not overhead from making assignments.

In [ ]:
print(C[50,50])
print(D[50,50])
print(E[50,50])

## Using Other Languages

Maybe some LaTEX.

In [ ]:
%%latex
\begin{equation}
\left. \begin{array}{lllll}
\lambda_{11} & = & \cos(x'_1,x_1) & = & \cos\theta \\
\lambda_{12} & = & \cos(x'_1,x_2) & = & \cos \bigg( \frac{\pi}{2} - \theta \bigg) = \sin\theta \\
\lambda_{21} & = & \cos(x'_2,x_1) & = & \cos \bigg( \frac{\pi}{2} + \theta \bigg) = - \sin\theta \\
\lambda_{22} & = & \cos(x'_2,x_2) & = & \cos\theta
\end{array} \right\}
\end{equation}

Perl, because we can.

In [ ]:
%%perl

$size=15;             # give $size value of 15
$y = -7.78;           # give $y value of -7.78
$z = 6 + $size;
print $y, "\n";
print $z, "\n";
print $size-$y, "\n";
$num = 7;
$txt = "It is $num";
print $txt;

## Repetition
Maybe we need to do a particular selective print, or some other repetitive action. Rather than defining a function for it, a macro can be defined relative to something in the history, such as the last action you took. Say we're interested in particular matrix elements that change. Print them, define a macro relative to the last action, then just use the macro.

In [ ]:
print(A[0,0], B[3,10])

In [ ]:
%hist -n -l 1

In [ ]:
%macro myprint 43

In [ ]:
myprint